<a href="https://colab.research.google.com/github/srilamaiti/spring_2023_w266_final_project_heesuk_iris_srila/blob/main/%20iris/IL_data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/competitions/feedback-prize-english-language-learning

In [1]:
# # install
# !pip install kaggle

In [ ]:
import pandas as pd
import numpy as np
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import tensorflow as tf
import os

In [ ]:
# data
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

%cd "gdrive/MyDrive/Colab Notebooks/"

In [ ]:
## only needed to run it once to download

# os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Colab Notebooks/"

# !kaggle competitions download -c feedback-prize-english-language-learning 
# !unzip -q feedback-prize-english-language-learning.zip -d .

In [ ]:
train = pd.read_csv("train.csv")
print(f"train shape: {train.shape}")
train.head()

In [ ]:
test = pd.read_csv("test.csv")
print(f"test shape: {test.shape}")
test.head()

In [ ]:
train.info()

In [ ]:
# length of full_text
pprint.pprint(train["full_text"][0])

In [ ]:
len(train["full_text"][0])

In [ ]:
# histogram with cohesion, syntax, vocabulary, phraseology, grammar, conventions 
for y in ["cohesion","syntax","vocabulary", "phraseology","grammar", "conventions"]:
  print(train[y].value_counts().sort_index())
  print()
  train[y].hist()
  plt.show()

In [ ]:
# any collinearity?
cm = train.corr()
print(cm)
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(cm, cmap=cmap)
plt.show()
# high correlation for all of them, especially phraseology+vocabulary

In [ ]:
# Any patterns of text? Head & tail has same topics? # range of topics with multiple paragraphs. pprint?
[print(train["full_text"][i]) for i in range(5)]

In [ ]:
# [print(train["full_text"][i]) for i in range(-5,-1,-1)]
train["full_text"][train.shape[1]]

In [ ]:
# any pattern for those who are high scorers in all of them?
# ["cohesion","syntax","vocabulary", "phraseology","grammar", "conventions"]
pd.set_option('display.max_colwidth', None)
# high scorer for one is high scorer for the rest.
train[(train["cohesion"]==5) & (train["syntax"]==5) & (train["vocabulary"]==5) & (train["phraseology"]==5) & (train["grammar"]==5) & (train["conventions"]==5)]


In [ ]:
# any pattern for those who are low scorers in all of them?
# ["cohesion","syntax","vocabulary", "phraseology","grammar", "conventions"]

# low scorer for one is low scorer for the rest.
train[(train["cohesion"]==1) & (train["syntax"]==1) & (train["vocabulary"]==1) & (train["phraseology"]==1) & (train["grammar"]==1) & (train["conventions"]==1)]


Notes:

* test.csv only had 3 items. just discard and split by train.csv
* high correlation, so high scorer in one is high scorer in rest and vice versa
* hard to be any extreme
* high scorers also tend to write more
* low scorers have a lot of misspellings, suggesting a model with more casual language might be more appropriate
* there are paragraphs so we need to take note of line separations


# RetriBERT

In [ ]:
# RetriBERT tokenizer
# class transformers.RetriBertTokenizer


In [ ]:
!pip install transformers
!pip install ktrain

In [ ]:
import math
from typing import Optional
import transformers
from ktrain import text

import torch
import torch.utils.checkpoint as checkpoint
from torch import nn
# from transformers import TFAutoModel, AutoTokenizer

# from ...modeling_utils import PreTrainedModel
# from ...utils import add_start_docstrings, logging
# from ..bert.modeling_bert import BertModel
# from .configuration_retribert import RetriBertConfig

In [ ]:
# model_checkpoint = 'yjernite/retribert-base-uncased'
# model = TFAutoModel.from_pretrained(model_checkpoint)

In [ ]:
# # RetriBERT model
# # class transformers.RetriBertModel
# # coding=utf-8
# # Copyright 2019-present, the HuggingFace Inc. team, The Google AI Language Team and Facebook, Inc.
# #
# # Licensed under the Apache License, Version 2.0 (the "License");
# # you may not use this file except in compliance with the License.
# # You may obtain a copy of the License at
# #
# #     http://www.apache.org/licenses/LICENSE-2.0
# #
# # Unless required by applicable law or agreed to in writing, software
# # distributed under the License is distributed on an "AS IS" BASIS,
# # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# # See the License for the specific language governing permissions and
# # limitations under the License.
# """
# RetriBERT model
# """

# logger = transformers.logging.get_logger(__name__)

# RETRIBERT_PRETRAINED_MODEL_ARCHIVE_LIST = [
#     "yjernite/retribert-base-uncased",
#     # See all RetriBert models at https://huggingface.co/models?filter=retribert
# ]


# # INTERFACE FOR ENCODER AND TASK SPECIFIC MODEL #
# class RetriBertPreTrainedModel(PreTrainedModel):
#     """
#     An abstract class to handle weights initialization and a simple interface for downloading and loading pretrained
#     models.
#     """

#     config_class = RetriBertConfig
#     load_tf_weights = None
#     base_model_prefix = "retribert"

#     def _init_weights(self, module):
#         """Initialize the weights"""
#         if isinstance(module, nn.Linear):
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#             if module.bias is not None:
#                 module.bias.data.zero_()
#         elif isinstance(module, nn.Embedding):
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#             if module.padding_idx is not None:
#                 module.weight.data[module.padding_idx].zero_()
#         elif isinstance(module, nn.LayerNorm):
#             module.bias.data.zero_()
#             module.weight.data.fill_(1.0)


# RETRIBERT_START_DOCSTRING = r"""
#     This model inherits from [`PreTrainedModel`]. Check the superclass documentation for the generic methods the
#     library implements for all its model (such as downloading or saving, resizing the input embeddings, pruning heads
#     etc.)
#     This model is also a PyTorch [torch.nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) subclass.
#     Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to general usage
#     and behavior.
#     Parameters:
#         config ([`RetriBertConfig`]): Model configuration class with all the parameters of the model.
#             Initializing with a config file does not load the weights associated with the model, only the
#             configuration. Check out the [`~PreTrainedModel.from_pretrained`] method to load the model weights.
# """


# @add_start_docstrings(
#     """Bert Based model to embed queries or document for document retrieval.""",
#     RETRIBERT_START_DOCSTRING,
# )
# class RetriBertModel(RetriBertPreTrainedModel):
#     def __init__(self, config: RetriBertConfig) -> None:
#         super().__init__(config)
#         self.projection_dim = config.projection_dim

#         self.bert_query = BertModel(config)
#         self.bert_doc = None if config.share_encoders else BertModel(config)
#         self.dropout = nn.Dropout(config.hidden_dropout_prob)
#         self.project_query = nn.Linear(config.hidden_size, config.projection_dim, bias=False)
#         self.project_doc = nn.Linear(config.hidden_size, config.projection_dim, bias=False)

#         self.ce_loss = nn.CrossEntropyLoss(reduction="mean")

#         # Initialize weights and apply final processing
#         self.post_init()

#     def embed_sentences_checkpointed(
#         self,
#         input_ids,
#         attention_mask,
#         sent_encoder,
#         checkpoint_batch_size=-1,
#     ):
#         # reproduces BERT forward pass with checkpointing
#         if checkpoint_batch_size < 0 or input_ids.shape[0] < checkpoint_batch_size:
#             return sent_encoder(input_ids, attention_mask=attention_mask)[1]
#         else:
#             # prepare implicit variables
#             device = input_ids.device
#             input_shape = input_ids.size()
#             token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)
#             head_mask = [None] * sent_encoder.config.num_hidden_layers
#             extended_attention_mask: torch.Tensor = sent_encoder.get_extended_attention_mask(
#                 attention_mask, input_shape
#             )

#             # define function for checkpointing
#             def partial_encode(*inputs):
#                 encoder_outputs = sent_encoder.encoder(
#                     inputs[0],
#                     attention_mask=inputs[1],
#                     head_mask=head_mask,
#                 )
#                 sequence_output = encoder_outputs[0]
#                 pooled_output = sent_encoder.pooler(sequence_output)
#                 return pooled_output

#             # run embedding layer on everything at once
#             embedding_output = sent_encoder.embeddings(
#                 input_ids=input_ids, position_ids=None, token_type_ids=token_type_ids, inputs_embeds=None
#             )
#             # run encoding and pooling on one mini-batch at a time
#             pooled_output_list = []
#             for b in range(math.ceil(input_ids.shape[0] / checkpoint_batch_size)):
#                 b_embedding_output = embedding_output[b * checkpoint_batch_size : (b + 1) * checkpoint_batch_size]
#                 b_attention_mask = extended_attention_mask[b * checkpoint_batch_size : (b + 1) * checkpoint_batch_size]
#                 pooled_output = checkpoint.checkpoint(partial_encode, b_embedding_output, b_attention_mask)
#                 pooled_output_list.append(pooled_output)
#             return torch.cat(pooled_output_list, dim=0)

#     def embed_questions(
#         self,
#         input_ids,
#         attention_mask=None,
#         checkpoint_batch_size=-1,
#     ):
#         q_reps = self.embed_sentences_checkpointed(
#             input_ids,
#             attention_mask,
#             self.bert_query,
#             checkpoint_batch_size,
#         )
#         return self.project_query(q_reps)

#     def embed_answers(
#         self,
#         input_ids,
#         attention_mask=None,
#         checkpoint_batch_size=-1,
#     ):
#         a_reps = self.embed_sentences_checkpointed(
#             input_ids,
#             attention_mask,
#             self.bert_query if self.bert_doc is None else self.bert_doc,
#             checkpoint_batch_size,
#         )
#         return self.project_doc(a_reps)

#     def forward(
#         self,
#         input_ids_query: torch.LongTensor,
#         attention_mask_query: Optional[torch.FloatTensor],
#         input_ids_doc: torch.LongTensor,
#         attention_mask_doc: Optional[torch.FloatTensor],
#         checkpoint_batch_size: int = -1,
#     ) -> torch.FloatTensor:
#         r"""
#         Args:
#             input_ids_query (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
#                 Indices of input sequence tokens in the vocabulary for the queries in a batch.
#                 Indices can be obtained using [`AutoTokenizer`]. See [`PreTrainedTokenizer.encode`] and
#                 [`PreTrainedTokenizer.__call__`] for details.
#                 [What are input IDs?](../glossary#input-ids)
#             attention_mask_query (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, *optional*):
#                 Mask to avoid performing attention on padding token indices. Mask values selected in `[0, 1]`:
#                 - 1 for tokens that are **not masked**,
#                 - 0 for tokens that are **masked**.
#                 [What are attention masks?](../glossary#attention-mask)
#             input_ids_doc (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
#                 Indices of input sequence tokens in the vocabulary for the documents in a batch.
#             attention_mask_doc (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, *optional*):
#                 Mask to avoid performing attention on documents padding token indices.
#             checkpoint_batch_size (`int`, *optional*, defaults to `-1`):
#                 If greater than 0, uses gradient checkpointing to only compute sequence representation on
#                 `checkpoint_batch_size` examples at a time on the GPU. All query representations are still compared to
#                 all document representations in the batch.
#         Return:
#             `torch.FloatTensor``: The bidirectional cross-entropy loss obtained while trying to match each query to its
#             corresponding document and each document to its corresponding query in the batch
#         """
#         device = input_ids_query.device
#         q_reps = self.embed_questions(input_ids_query, attention_mask_query, checkpoint_batch_size)
#         a_reps = self.embed_answers(input_ids_doc, attention_mask_doc, checkpoint_batch_size)
#         compare_scores = torch.mm(q_reps, a_reps.t())
#         loss_qa = self.ce_loss(compare_scores, torch.arange(compare_scores.shape[1]).to(device))
#         loss_aq = self.ce_loss(compare_scores.t(), torch.arange(compare_scores.shape[0]).to(device))
#         loss = (loss_qa + loss_aq) / 2
#         return loss

In [ ]:
# retribert_model = transformers.TFAutoModel.from_pretrained("yjernite/retribert-base-uncased")
# text.Transformer("yjernite/retribert-base-uncased/")
# transformers.AutoConfig.from_pretrained("yjernite/retribert-base-uncased/")

# retriconfig = transformers.RetriBertConfig(vocab_size=30522, 
#                              hidden_size=768, 
#                              num_hidden_layers=8, 
#                              num_attention_heads=12, 
#                              intermediate_size=3072, 
#                              hidden_act='gelu', 
#                              hidden_dropout_prob=0.1, 
#                              attention_probs_dropout_prob=0.1, 
#                              max_position_embeddings=512, 
#                              type_vocab_size=2, 
#                              initializer_range=0.02, 
#                              layer_norm_eps=1e-12, 
#                              share_encoders=True, 
#                              projection_dim=128, 
#                              pad_token_id=0)
# # retribert_model = transformers.TFAutoModel.from_pretrained(retriconfig)
# transformers.AutoConfig.from_pretrained(retriconfig) # tried using the config function

retribert_model_checkpoints = "yjernite/retribert-base-uncased"
retribert_model = transformers.BertModel.from_pretrained(retribert_model_checkpoints)
# retribert_model

# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
for param in retribert_model.parameters():
    param.requires_grad = False

retribert_tokenizer = transformers.BertTokenizer.from_pretrained(retribert_model_checkpoints)
# retribert_tokenizer = transformers.AutoTokenizer.from_pretrained(retribert_model_checkpoints, use_fast=False, normalization = True)

In [ ]:
retri_cuda = retribert_model.cuda()
retri_cuda

In [ ]:
input_train_df = train
input_test_df = test
label_cols = input_train_df.columns[2:]
orig_train_df = copy.deepcopy(input_train_df)
orig_train_df.head()
     
shuffle = np.random.permutation(np.arange(orig_train_df.shape[0]))
orig_train_df = orig_train_df.iloc[shuffle]
split=(0.8,0.1,0.1)
splits = np.multiply(len(orig_train_df), split).astype(int)
df_train, df_val, df_test = np.split(orig_train_df, [splits[0], splits[0] + splits[1]])

X_train, X_val, X_test = df_train['full_text'], df_val['full_text'], df_test['full_text']
y_train, y_val, y_test = np.array(df_train[label_cols]), np.array(df_val[label_cols]), np.array(df_test[label_cols])

In [ ]:
def set_config_param(seed = 99):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.keras.backend.clear_session()
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle"
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_columns', None)
    
    
# set_config_param(20230214)
set_config_param()

In [ ]:
MAX_LENGTH = 512
epochs = 5
batch_size = 8
dropout = .1
learning_rate = .00005
number_of_hidden_layer = 1
hidden_layer_node_count = 256
trainable_flag = False
retrain_layer_count = 0

In [ ]:
def MCRMSE(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=-1, keepdims=True)

def plot_loss_accuracy(history, col_list):
    fig, ax = plt.subplots(2, 6, figsize=(16, 6), sharex='col', sharey='row')
    fig.tight_layout(pad=5.0)
    for idx, col in enumerate(col_list):

        ax[0, idx].plot(history[col + '_loss'], lw=2, color='darkgoldenrod')
        ax[0, idx].plot(history['val_' + col + '_loss'], lw=2, color='indianred')
        #ax[0, idx].legend(loc='center left')
        ax[0, idx].legend(['Train', 'Validation'], fontsize=5)
        ax[0, idx].set_xlabel('Epochs', size=10)
        ax[0, idx].set_title('Loss: ' + col)

        ax[1, idx].plot(history[col + '_accuracy'], lw=2, color='darkgoldenrod')
        ax[1, idx].plot(history['val_' + col + '_accuracy'], lw=2, color='indianred')
        #ax[0, idx].legend(loc='center left')
        ax[1, idx].legend(['Train', 'Validation'], fontsize=5)
        ax[1, idx].set_xlabel('Epochs', size=10)
        ax[1, idx].set_title('Accuracy: ' + col)

def encode_text(text, tokenizer):
    
    encoded = tokenizer.batch_encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors="tf",
    )

    input_ids = np.array(encoded["input_ids"], dtype="int32")
    attention_masks = np.array(encoded["attention_mask"], dtype="int32")

    return {
        "input_ids": input_ids,
        "attention_masks": attention_masks
    }

In [ ]:
train_encodings = encode_text(df_train['full_text'].tolist(), retribert_tokenizer)
val_encodings = encode_text(df_val['full_text'].tolist(), retribert_tokenizer)
test_encodings = encode_text(df_test['full_text'].tolist(), retribert_tokenizer)

In [ ]:
# double check encodings and length=512

In [ ]:
#### from BERT TF
# def create_retribert_classification_model(retribert_model,
#                                           num_train_layers=0,
#                                           hidden_size = 200,
#                                           dropout=0.3,
#                                           learning_rate=0.00005):
#     """
#     Build a simple classification model with BERT. Use the CLS Output for classification purposes
#     """
#     if num_train_layers == 0:
#         # Freeze all layers of pre-trained BERT model
#         retribert_model.trainable = False

#     elif num_train_layers == 12: ##### NUMBER OF RETRIBERT MODELS. NEED TO UPDATE.
#         # Train all layers of the BERT model
#         retribert_model.trainable = True

#     else:
#         # Restrict training to the num_train_layers outer transformer layers
#         retrain_layers = []

#         for retrain_layer_number in range(num_train_layers):

#             layer_code = '_' + str(11 - retrain_layer_number)
#             retrain_layers.append(layer_code)
          
        
#         print('retrain layers: ', retrain_layers)

#         for w in retribert_model.weights:
#             if not any([x in w.name for x in retrain_layers]):
#                 #print('freezing: ', w)
#                 w._trainable = False

#     input_ids = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int64, name='input_ids_layer')
#     token_type_ids = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int64, name='token_type_ids_layer')
#     attention_mask = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int64, name='attention_mask_layer')

#     retribert_inputs = {'input_ids': input_ids,
#                         'token_type_ids': token_type_ids,
#                         'attention_mask': attention_mask}      

#     retribert_out = retribert_model(retribert_inputs)

#     cls_token = retribert_out[0][:, 0, :]

#     hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)


#     hidden = tf.keras.layers.Dropout(dropout)(hidden)


#     classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)
    
#     classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
    
#     classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#                                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
#                                  metrics='accuracy')
    
#     return classification_model

In [ ]:
# retribert_classification_model = create_retribert_classification_model(retribert_model, num_train_layers=0)
# retribert_classification_model.summary()

In [ ]:
# Import required libraries/code
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from torch import nn
from torchvision import transforms, datasets

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine
     


In [ ]:
# from torch.utils.data import Dataset

# class PandasDataset(Dataset):
#     def __init__(self, dataframe):
#         self.dataframe = dataframe

#     def __len__(self):
#         return len(self.dataframe)

#     def __getitem__(self, index):
#         return self.dataframe.iloc[index]

# from torch.utils.data import DataLoader
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(orig_train_df, test_size=0.2, random_state=0)
# train_dataset = PandasDataset(train)
# test_dataset = PandasDataset(test)
# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16)
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16)

# train_dataloader = torch.utils.data.DataLoader(X_train, batch_size=64, shuffle=True)
# test_dataloader = torch.utils.data.DataLoader(X_test, batch_size=64, shuffle=True)

In [ ]:
# # Set the manual seeds
# torch.manual_seed(42)
# torch.cuda.manual_seed(42)
# class_names = [1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0]
# # Get the length of class_names (one output unit for each class)
# output_shape = len(class_names)


# # Recreate the classifier layer and seed it to the target device
# retribert_model.classifier = torch.nn.Sequential(
#     torch.nn.Dropout(p=0.2, inplace=True), 
#     torch.nn.Linear(in_features=1280, 
#                     out_features=output_shape, # same number of output units as our number of classes
#                     bias=True)).to(device)


# # Define loss and optimizer
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(retribert_model.parameters(), lr=0.001)

# # retribert_model_results = engine.train(model=retribert_model,
# #                                        train_dataloader=train_dataloader,
# #                                        test_dataloader=test_dataloader,
# #                                        optimizer=optimizer,
# #                                        loss_fn=loss_fn,
# #                                        epochs=5,
# #                                        device=device)

In [ ]:
import tqdm

In [ ]:
train_encodings

In [ ]:
## copy of code above
X_train, X_val, X_test = df_train['full_text'], df_val['full_text'], df_test['full_text']
y_train, y_val, y_test = np.array(df_train[label_cols]), np.array(df_val[label_cols]), np.array(df_test[label_cols])

#### from
# https://stackoverflow.com/questions/55369821/how-to-train-a-neural-network-model-with-bert-embeddings-instead-of-static-embed
retribert_tokenizer = transformers.BertTokenizer.from_pretrained(retribert_model_checkpoints)
# retribert_tokenizer = transformers.AutoTokenizer.from_pretrained(retribert_model_checkpoints, use_fast=False, normalization = True)

X_train = [retribert_tokenizer.tokenize('[CLS] ' + sent + ' [SEP]') for sent in X_train] 
X_train_tokens = [retribert_tokenizer.convert_tokens_to_ids(sent) for sent in X_train]
# # X_train[0]
# # X_train_tokens[0]
X_test = [retribert_tokenizer.tokenize('[CLS] ' + sent + ' [SEP]') for sent in X_test] 
X_test_tokens = [retribert_tokenizer.convert_tokens_to_ids(sent) for sent in X_test]
# y_train_tensor = torch.from_numpy(y_train)

In [ ]:
### train individual tensors
y_train_cohesion = [y_train[i][0] for i in range(len(y_train))]
y_train_syntax = [y_train[i][1] for i in range(len(y_train))]
y_train_vocabulary = [y_train[i][2] for i in range(len(y_train))]
y_train_phraseology = [y_train[i][3] for i in range(len(y_train))]
y_train_grammar = [y_train[i][4] for i in range(len(y_train))]
y_train_convention = [y_train[i][5] for i in range(len(y_train))]

y_train_cohesion_tensor = torch.Tensor(y_train_cohesion)

In [ ]:
# ## trying with dataloader
# # len(X_train_tokens) #3128
# # len(y_train_cohesion) #33128
# cohesion_dict = {"X_train_tokens":X_train_tokens,
#                  "y_train_cohesion": y_train_cohesion}
# cohesion_df = pd.DataFrame(cohesion_dict)

# class customDataset:
#   def __init__(self):
#     xy = cohesion_df
#     self.len = xy.shape[0]
#     self.x_data = torch.from_numpy(xy[:,0:-1])
#     self.y_data = torch.from_numpy(xy.iloc[:,[-1]])
#   def __getitem__(self,index):
#     return self.x_data[index], self.y_data[index]
#   def __len__(self):
#     return self.len

# cohesion_ds = customDataset()
# training_data = torchvision.datasets.cohesion_ds(
#     root="data",
#     train=True,
#     download=True,
#     transform=torchvision.transformsToTensor()
# )

# # train_dataloader = torch.utils.data.DataLoader(cohesion_df, batch_size=8, shuffle=True)
# # train_dataloader

# # train_features, train_labels = iter(train_dataloader)
# # print(f"Feature batch shape: {train_features.size()}")
# # print(f"Labels batch shape: {train_labels.size()}")
# # img = train_features[0].squeeze()
# # label = train_labels[0]
# # plt.imshow(img, cmap="gray")
# # plt.show()
# # print(f"Label: {label}")
# # test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

class pyDataset(torch.utils.data.Dataset):
  def __init__(self, X,y):
    self.X = X
    self.y = y
  def __len__(self):
    return len(self.X)
  def __getitem__(self,idx):
    return self.X[idx], self.y[idx]
train_loader = torch.utils.data.DataLoader(dataset=pyDataset(X_train, y_train), batch_size=8, shuffle=T)

In [ ]:
# https://stackoverflow.com/questions/64156202/add-dense-layer-on-top-of-huggingface-bert-model
### this seems to be working  but out of memory
class CustomBERTModel(nn.Module):
    def __init__(self):
          super(CustomBERTModel, self).__init__()
          self.retribert = transformers.BertModel.from_pretrained(retribert_model_checkpoints)
          # for param in self.retribert.parameters():
            # param.requires_grad = False
          self.linear1 = nn.Linear(768, 9)
          self.dropout1 = nn.Dropout(0.1)

    def forward(self, ids, mask):
          outputs = self.retribert(ids,attention_mask=mask)
          last_hidden_state = outputs[0]

          # sequence_output has the following shape: (batch_size, sequence_length, 768)
          linear1_output = self.linear1(last_hidden_state[:,0,:].view(-1,768)) ## extract the 1st token's embeddings
          # ### New layers:
          linear2_dropout_output = self.dropout1(linear1_output)

          return linear2_dropout_output

retribert_model = CustomBERTModel() # You can pass the parameters if required to have more flexible model
retribert_model.to(torch.device("cuda")) ## can be gpu
criterion = nn.CrossEntropyLoss() ## If required define your own criterion
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, retribert_model.parameters()))

for epoch in range(5):
  optimizer.zero_grad()
  predicted = []
  for sent in X_train:
    encoded_sent = retribert_tokenizer.encode_plus(sent, 
                                                   add_special_tokens=True, 
                                                   truncation=True, 
                                                   padding=True, 
                                                   return_attention_mask=True, 
                                                   return_tensors='pt', 
                                                   max_length=510)
    outputs_1 = retribert_model(encoded_sent["input_ids"].to("cuda"), encoded_sent["attention_mask"].to("cuda"))
    outputs_1
  #   final_outputs = torch.nn.functional.log_softmax(outputs_1, dim=1)
  # # print(final_outputs)
  #   final_outputs_max = torch.max(final_outputs)
  #   predicted.append(final_outputs_max)
  # print(predicted)
  # print(final_outputs_2)
        # input_ids = encoding['input_ids']
        # attention_mask = encoding['attention_mask']
  # loss = criterion(final_outputs, targets)
        # loss.backward()
        # optimizer.step()


# for epoch in epochs:
#     for batch in data_loader: ## If you have a DataLoader()  object to get the data.

#         data = batch[0]
#         targets = batch[1] ## assuming that data loader returns a tuple of data and its targets
        
#         optimizer.zero_grad()   
#         encoding = tokenizer.batch_encode_plus(data, return_tensors='pt', padding=True, truncation=True,max_length=510, add_special_tokens = True)
        # outputs = model(input_ids, attention_mask=attention_mask)
        # outputs = F.log_softmax(outputs, dim=1)
        # input_ids = encoding['input_ids']
        # attention_mask = encoding['attention_mask']
        # loss = criterion(outputs, targets)
        # loss.backward()
        # optimizer.step()

In [ ]:
# use MCRMSE as loss metric

# def MCRMSE(y_true, y_pred):
#     colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis = 1)
#     return tf.reduce_mean(tf.sqrt(colwise_mse), axis = -1, keepdims = True)
     


In [ ]:
### anothing thing
### https://stackoverflow.com/questions/68115993/input-ids-torch-tensorinput-ids-valueerror-expected-sequence-of-length-133
input_ids = []
attention_masks = []
# print(flaubert)
for sent in X_train:
    encoded_sent = retribert_tokenizer.encode_plus(sent, add_special_tokens=True, truncation=True, padding=True, return_attention_mask=True, return_tensors='pt', max_length=510)

    # Add the outputs to the lists
    # input_ids.append(encoded_sent.get('input_ids'))
    # attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors

# input_ids = torch.as_tensor(input_ids)
# attention_masks = torch.as_tensor(attention_masks)

hidden_state = retribert_model(input_ids=encoded_sent["input_ids"].to("cuda"), attention_mask=encoded_sent["attention_mask"].to("cuda"))
hidden_state
# # # Extract the last hidden state of the token `[CLS]` for classification task
last_hidden_state_cls = hidden_state[0][:, 0, :]

retribert_model.linear1 = nn.Linear(768, 256)
retribert_model.dropout1 = nn.Dropout(0.1)
# device = "cuda" if torch.cuda.is_available() else "cpu"
# device

In [ ]:
list(retribert_model.parameters())[0] # model weights?

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0186, -0.0166, -0.0154,  ...,  0.0220,  0.0171,  0.0076],
        [ 0.0118,  0.0026,  0.0049,  ..., -0.0287,  0.0192,  0.0038],
        ...,
        [-0.0003, -0.0193, -0.0029,  ..., -0.0182, -0.0184, -0.0021],
        [ 0.0355, -0.0214, -0.0174,  ..., -0.0083,  0.0015,  0.0047],
        [-0.0179,  0.0016,  0.0094,  ..., -0.0528,  0.0078,  0.0255]],
       device='cuda:0')

In [ ]:
# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# # Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
# for param in retribert_model.parameters():
#     param.requires_grad = False

# retribert_model(last_hidden_state_cls)

# ## from https://pypi.org/project/pytorch-pretrained-bert/

# with torch.no_grad():
#     predictions = retribert_model(last_hidden_state_cls)

RuntimeError: ignored

In [ ]:
# results = torch.zeros((len(X_test_tokens), retri_cuda.config.hidden_size)).long()
# results[0]
# with torch.no_grad():
#     for stidx in range(0, len(X_test_tokens), batch_size):
#         X = X_test_tokens[stidx:stidx + batch_size]
#         X = torch.LongTensor(X).cuda()
#         embed, pooled_output = retri_cuda(X)
#         results[stidx:stidx + batch_size,:] = embed.cpu()
# torch.LongTensor(X)

#### from https://machinelearningmastery.com/building-a-regression-model-in-pytorch/
# X_train_torch = torch.tensor(X_train_tokens, dtype=torch.float32)
# y_train_torch = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
# X_test_torch = torch.tensor(X_test, dtype=torch.float32)
# y_test_torch = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
 
# # training parameters
# n_epochs = 5   # number of epochs to run
# batch_size = 8  # size of each batch
# batch_start = torch.arange(0, len(X_train), batch_size)
 
# # Hold the best model
# best_mse = np.inf   # init to infinity
# best_weights = None
# history = []
 
# # training loop
# for epoch in range(n_epochs):
#     retribert_model.train()
#     with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
#         bar.set_description(f"Epoch {epoch}")
#         for start in bar:
#             # take a batch
#             X_batch = X_train[start:start+batch_size]
#             y_batch = y_train[start:start+batch_size]
#             # forward pass
#             y_pred = retribert_model(X_batch)
#             loss = loss_fn(y_pred, y_batch)
#             # backward pass
#             optimizer.zero_grad()
#             loss.backward()
#             # update weights
#             optimizer.step()
#             # print progress
#             bar.set_postfix(mse=float(loss))
#     # evaluate accuracy at end of each epoch
#     retribert_model.eval()
#     y_pred = retribert_model(X_test)
#     mse = loss_fn(y_pred, y_test)
#     mse = float(mse)
#     history.append(mse)
#     if mse < best_mse:
#         best_mse = mse
#         best_weights = copy.deepcopy(retribert_model.state_dict())
 
# # restore model and return best accuracy
# retribert_model.load_state_dict(best_weights)

In [ ]:
### evaluating the model
# from https://discuss.pytorch.org/t/multi-class-bert-model-class-weights-and-where-to-use-them/176264
# def evaluate():  
#     model.eval()
#     total_loss, total_accuracy = 0, 0
#     total_preds = []
#     for step,batch in enumerate(val_dataloader):
#         if step % 50 == 0 and not step == 0:
#             elapsed = format_time(time.time() - t0)
#             print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))
#         batch = [t.to(device) for t in batch]
#         sent_id, mask, labels = batch
#         with torch.no_grad():
#             preds = model(sent_id, mask)
#             loss = cross_entropy(preds,labels)  #<------- Here
#             total_loss = total_loss + loss.item()
#             preds = preds.detach().cpu().numpy()
#             total_preds.append(preds)
#     avg_loss = total_loss / len(val_dataloader) 
#     total_preds  = np.concatenate(total_preds, axis=0)
#     return avg_loss, total_preds

Converting to TensorFlow?

In [ ]:
# !pip install tensorflow
# !pip install tensorflow-addons
# !pip install onnx
# !git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow && pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'onnx-tensorflow' already exists and is not an empty directory.


In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import datasets, transforms
# from torch.autograd import Variable
# import onnx
# from onnx_tf.backend import prepare

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
# retribert_model = transformers.BertModel.from_pretrained(retribert_model_checkpoints)
# torch.save(retribert_model.state_dict(), './retribert.pt')

In [ ]:
# model_pytorch = retribert_model()
# model_pytorch.load_state_dict('./retribert.pt')

# dummy_input = torch.from_numpy(X_train_tokens.to("cuda"))
# dummy_output = model_pytorch(dummy_input)

ValueError: ignored